In [ ]:
import os
from pathlib import Path
import pypdf
from docx import Document as DocxDocument
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.memory import ConversationBufferMemory
from agno.agent import Agent

# Azure OpenAI Configuration
AZURE_CONFIG = {
    "api_key": "",
    "endpoint": "",
    "api_version": "2024-12-01-preview",
    "embedding_deployment": "text-embedding-ada-002",
    "gpt_deployment": "gpt-4o"
}

# Initialize Azure OpenAI clients
llm = AzureChatOpenAI(
    openai_api_key=AZURE_CONFIG["api_key"],
    azure_endpoint=AZURE_CONFIG["endpoint"],
    api_version=AZURE_CONFIG["api_version"],
    deployment_name=AZURE_CONFIG["gpt_deployment"],
    temperature=0.7
)

embeddings = AzureOpenAIEmbeddings(
    openai_api_key=AZURE_CONFIG["api_key"],
    azure_endpoint=AZURE_CONFIG["endpoint"],
    api_version=AZURE_CONFIG["api_version"],
    deployment=AZURE_CONFIG["embedding_deployment"]
)
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:

# InsuranceRAGSystem: Manages document ingestion, storage, and retrieval
class InsuranceRAGSystem:
    def __init__(self, data_dir="data", db_dir="chroma_db"):
        self.data_dir = Path(data_dir)
        self.db_dir = Path(db_dir)
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        self.vectorstore = None
        self.initialize_data_dir()
        self.initialize_vectorstore()

    def initialize_data_dir(self):
        """Create data directory and sample document if empty."""
        if not self.data_dir.exists():
            self.data_dir.mkdir()
            self.create_sample_document()
        elif not any(self.data_dir.iterdir()):
            self.create_sample_document()

    def create_sample_document(self):
        """Create a sample insurance document if none exist."""
        sample_content = """
        # Insurance Basics
        ## Auto Insurance Types
        - **Liability Coverage**: Covers damages to others if you're at fault in an accident.
        - **Collision Coverage**: Covers damage to your car from a collision.
        - **Comprehensive Coverage**: Covers non-collision damage (e.g., theft, natural disasters).
        - **Personal Injury Protection (PIP)**: Covers medical expenses for you and your passengers.
        ## Home Insurance Claims
        - **Step 1**: Contact your insurance provider immediately.
        - **Step 2**: Document the damage with photos and videos.
        - **Step 3**: Submit a claim form with detailed descriptions.
        """
        with open(self.data_dir / "sample_insurance_info.txt", "w") as f:
            f.write(sample_content)

    def load_documents(self):
        """Load and process documents from data directory."""
        documents = []
        for file_path in self.data_dir.iterdir():
            if file_path.suffix.lower() in [".pdf", ".txt", ".docx"]:
                content = self.read_file(file_path)
                if content:
                    doc = Document(page_content=content, metadata={"source": str(file_path)})
                    documents.append(doc)
        return documents

    def read_file(self, file_path):
        """Read content from PDF, text, or Word files."""
        try:
            if file_path.suffix.lower() == ".pdf":
                with open(file_path, "rb") as f:
                    pdf = pypdf.PdfReader(f)
                    return "".join(page.extract_text() for page in pdf.pages if page.extract_text())
            elif file_path.suffix.lower() == ".txt":
                with open(file_path, "r", encoding="utf-8") as f:
                    return f.read()
            elif file_path.suffix.lower() == ".docx":
                doc = DocxDocument(file_path)
                return "\n".join(paragraph.text for paragraph in doc.paragraphs if paragraph.text)
            return ""
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            return ""

    def initialize_vectorstore(self):
        """Initialize ChromaDB vectorstore with documents."""
        documents = self.load_documents()
        if not documents:
            self.create_sample_document()
            documents = self.load_documents()
        
        chunks = self.text_splitter.split_documents(documents)
        self.vectorstore = Chroma.from_documents(
            documents=chunks,
            embedding=embeddings,
            persist_directory=str(self.db_dir)
        )
        self.vectorstore.persist()

    def search(self, query):
        """Search for relevant documents based on query."""
        if not self.vectorstore:
            return []
        return self.vectorstore.similarity_search(query, k=3)

    def stats(self):
        """Return statistics about stored documents."""
        if not self.vectorstore:
            return "No documents loaded."
        return f"Number of documents: {self.vectorstore._collection.count()}"

    def reset(self):
        """Reset the vectorstore."""
        if self.db_dir.exists():
            for item in self.db_dir.iterdir():
                if item.is_file():
                    item.unlink()
                elif item.is_dir():
                    for subitem in item.iterdir():
                        subitem.unlink()
                    item.rmdir()
            self.db_dir.rmdir()
        self.initialize_vectorstore()

# InsuranceMultiAgentSystem: Manages collaborative agents
class InsuranceMultiAgentSystem:
    def __init__(self, rag_system, llm):
        self.rag_system = rag_system
        self.llm = llm
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        self.initialize_agents()

    def initialize_agents(self):
        """Initialize the team of agents using Agno."""
        # Define agent instructions
        knowledge_retriever_instructions = [
            "You are a KnowledgeRetriever. Use the provided context to answer the user's question.",
            "Focus on retrieving accurate information from insurance documents."
        ]
        claims_specialist_instructions = [
            "You are a ClaimsSpecialist. Provide detailed guidance on handling insurance claims.",
            "Explain steps clearly, including any documentation or processes required."
        ]
        policy_advisor_instructions = [
            "You are a PolicyAdvisor. Offer expert advice on insurance policy options.",
            "Explain policy types, coverage details, and considerations for choosing them."
        ]
        customer_service_instructions = [
            "You are a CustomerService agent. Greet the user warmly and coordinate responses.",
            "Ensure the user's question is clearly understood before passing it to other agents."
        ]
        lead_agent_instructions = [
            "You are a LeadAgent coordinating a team of specialized insurance agents.",
            "Collect responses from the KnowledgeRetriever, ClaimsSpecialist, PolicyAdvisor, and CustomerService agents.",
            "Summarize and refine their inputs into a clear, concise, and accurate final answer."
        ]

        # Initialize sub-agents
        self.knowledge_retriever = Agent(
            name="KnowledgeRetriever",
            model=self.llm,
            instructions=knowledge_retriever_instructions,
            description="Retrieves relevant information from insurance documents.",
            markdown=True
        )
        self.claims_specialist = Agent(
            name="ClaimsSpecialist",
            model=self.llm,
            instructions=claims_specialist_instructions,
            description="Specializes in insurance claim processes.",
            markdown=True
        )
        self.policy_advisor = Agent(
            name="PolicyAdvisor",
            model=self.llm,
            instructions=policy_advisor_instructions,
            description="Advises on insurance policy options.",
            markdown=True
        )
        self.customer_service = Agent(
            name="CustomerService",
            model=self.llm,
            instructions=customer_service_instructions,
            description="Coordinates responses and greets users.",
            markdown=True
        )

        # Initialize lead agent with team
        self.lead_agent = Agent(
            name="LeadAgent",
            model=self.llm,
            team=[self.knowledge_retriever, self.claims_specialist, self.policy_advisor, self.customer_service],
            instructions=lead_agent_instructions,
            description="Coordinates the insurance agent team and provides final responses.",
            markdown=True
        )

    def answer_question(self, question):
        """Process a question through the agent team."""
        # Retrieve relevant documents
        docs = self.rag_system.search(question)
        context = "\n".join([doc.page_content for doc in docs])
        
        # CustomerService initiates the conversation
        cs_response = self.llm.invoke(
            f"Greetings! I'm here to help with your insurance question: {question}"
        ).content
        
        # KnowledgeRetriever uses document context
        kr_response = self.llm.invoke(
            f"Question: {question}\nContext: {context}\nInstructions: {self.knowledge_retriever.instructions[0]}"
        ).content
        
        # ClaimsSpecialist and PolicyAdvisor provide specialized input
        cspecial_response = self.llm.invoke(
            f"Question: {question}\nInstructions: {self.claims_specialist.instructions[0]}"
        ).content
        pa_response = self.llm.invoke(
            f"Question: {question}\nInstructions: {self.policy_advisor.instructions[0]}"
        ).content
        
        # LeadAgent summarizes
        inputs = (
            f"CustomerService: {cs_response}\n"
            f"KnowledgeRetriever: {kr_response}\n"
            f"ClaimsSpecialist: {cspecial_response}\n"
            f"PolicyAdvisor: {pa_response}"
        )
        final_response = self.llm.invoke(
            f"Question: {question}\nInputs from team:\n{inputs}\nInstructions: {self.lead_agent.instructions[0]}"
        ).content
        
        # Update memory
        self.memory.save_context({"input": question}, {"output": final_response})
        
        return final_response, docs

# Main program
def main():
    # Initialize systems
    rag_system = InsuranceRAGSystem()
    multi_agent_system = InsuranceMultiAgentSystem(rag_system, llm)

    # Test with sample questions
    sample_questions = [
        "What types of auto insurance should I consider?",
        "How do I file a home insurance claim?"
    ]

    print("Testing with sample questions:")
    for question in sample_questions:
        print(f"\nQuestion: {question}")
        answer, docs = multi_agent_system.answer_question(question)
        print(f"Answer: {answer}")
        print("Relevant Documents:")
        for doc in docs:
            print(f"- {doc.metadata['source']}: {doc.page_content[:100]}...")

    # Interactive mode
    print("\nInteractive Mode - Type your question or use commands: stats, search [query], reload, reset, quit")
    while True:
        user_input = input("\nYour input: ").strip()
        if user_input.lower() == "quit":
            break
        elif user_input.lower() == "stats":
            print(rag_system.stats())
        elif user_input.lower().startswith("search "):
            query = user_input[7:].strip()
            docs = rag_system.search(query)
            print("Search Results:")
            for doc in docs:
                print(f"- {doc.metadata['source']}: {doc.page_content[:100]}...")
        elif user_input.lower() == "reload":
            rag_system.initialize_vectorstore()
            print("Documents reloaded.")
        elif user_input.lower() == "reset":
            rag_system.reset()
            print("Vectorstore reset.")
        else:
            answer, docs = multi_agent_system.answer_question(user_input)
            print(f"Answer: {answer}")
            print("Relevant Documents:")
            for doc in docs:
                print(f"- {doc.metadata['source']}: {doc.page_content[:100]}...")

if __name__ == "__main__":
    main()

INFO:httpx:HTTP Request: POST https://gptidk.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Testing with sample questions:

Question: What types of auto insurance should I consider?


INFO:httpx:HTTP Request: POST https://gptidk.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://gptidk.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://gptidk.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://gptidk.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://gptidk.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://gptidk.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Answer: As the LeadAgent, I appreciate the thorough input from the team. Each member has provided valuable insights into the key types of auto insurance coverage and their importance. To streamline this information and provide a cohesive response, I’ll consolidate and prioritize the recommendations, keeping in mind the customer’s potential concerns, such as state requirements, financial protection, and risk factors.

---

### **Types of Auto Insurance to Consider**
Here’s a comprehensive yet concise guide to help the customer make an informed decision:

---

### **1. Liability Coverage (Mandatory in Most States)**
   - **What It Is**: Covers bodily injury and property damage you cause to others.
   - **Why It’s Needed**: This is the legal minimum required in most states. It protects you from financial responsibility for injuries or damages caused to others.
   - **Expert Tip**: Opt for higher limits than the state minimum (e.g., $100,000/$300,000 for bodily injury and $50,000 for prope

INFO:httpx:HTTP Request: POST https://gptidk.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://gptidk.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://gptidk.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://gptidk.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://gptidk.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://gptidk.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Answer: As the LeadAgent, I’ll compile and streamline the inputs from the team into a cohesive guide for filing a home insurance claim. Here’s the step-by-step process, combining the insights from CustomerService, KnowledgeRetriever, ClaimsSpecialist, and PolicyAdvisor:

---

### **Step-by-Step Guide to Filing a Home Insurance Claim**

### **1. Assess the Damage or Loss**
- **Ensure Safety First**: Prioritize the safety of your family and property. If the situation involves hazards like fire, flooding, or structural damage, contact emergency services immediately.
- **Document the Damage**: Take clear photos and videos of the affected areas, damaged belongings, or signs of theft. The more detailed, the better.
- **Prevent Further Damage**: Take reasonable steps to avoid additional damage (e.g., covering a leaking roof or broken windows). Keep receipts for any temporary repairs, as these may be reimbursed.

---

### **2. Review Your Policy**
- **Understand Your Coverage**: Familiarize yo

INFO:openai._base_client:Retrying request to /embeddings in 0.475742 seconds
INFO:openai._base_client:Retrying request to /embeddings in 0.753161 seconds


APIConnectionError: Connection error.